In [3]:
import sys
assert sys.version_info >= (3, 5)

In [4]:
import sklearn
assert sklearn.__version__ >= "0.20"

In [5]:
import numpy as np
import os
import tarfile
import urllib
import pandas as pd
import urllib.request

In [6]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

Get the CSV data in to a panda dataframe, might need to alter filepaths depending where this is saved

In [ ]:
def load_data():
    return pd.read_csv("data/winemag-data-130k-v2.csv")

wine = load_data()
wine.head()

Ideas for pre-processing:

Create Vintage feature by scraping data from title field

Maybe scrape description for descriptive keywords?

Maybe try to find the longitude and latitude of the winery for some scattergraph data


Remove taster_twitter_handle

Remove Unnamed id feature

Drop the unneeded Id and taster twitter handle columns

In [231]:
wine = wine.drop(['Unnamed: 0', 'taster_twitter_handle'], axis=1)

Create a new feature for the vintage of the wine, get the data from the title field

Use RegEx to find the vintage year, make sure year is between 1975 and 2022 to avoid incorrect data. Some names don't contain vintage year, might need to remove those

In [8]:
import re

In [215]:
def getVintage(wine):
    f = re.findall(r'\d\d\d\d', wine["title"])
    if len(f) == 0:
        return int()
    elif len(f) > 1:
        for y in f:
            if int(y) < 1975 and int(y) > 2022:
                del f[y]
    f = f[0]
    return f

In [ ]:
wine["vintage"] = wine.apply(getVintage, axis=1)

Drop all records that have no indication of location

In [232]:
locCols = ["country", "province", "region_1"]
wine = wine.dropna(axis=0,subset=locCols, how='all', inplace=True)

This is a script that tries to get the longitude and latitude of each wineries location, it queries an online google maps esque webpage and gradually widens the search area until there is a match. It is not viable to get all of the data this way.

In [ ]:
%pip install requests

In [2]:
import requests
import urllib.parse

In [135]:
def getCoords(wine):
    address = str(wine["winery"]) + "," + str(wine["province"]) + "," + str(wine["region_1"]) + "," + str(wine["country"])
    repCount = 0
    while True:
        url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'
        response = requests.get(url).json()
        if response:
            print(response[0]["lat"])
            print(response[0]["lon"])
            return [response[0]["lat"], response[0]["lon"]]
        address = address.split(',')
        address = ','.join(address[1:])

If we let the code run, we should create some code to populate the longitude and latitude of the other records of each winery. At this current point, only the first record of each winery has their long and lat updated

In [ ]:
# Get the first record of each unique winery
uniqueWineries = wine.groupby("winery", as_index=False).first()

uniqueWineries["longitude"] = uniqueWineries.apply(getCoords, axis=1)[0]
uniqueWineries["latitude"] = uniqueWineries.apply(getCoords, axis=1)[1]